In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact, Dropdown

In [ ]:
data = pd.read_csv(r'C:\Users\UGBOKE GEORGE\Downloads\NationalNames\N_gram-name-prediction\NationalNames.csv', encoding= 'latin1')
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [ ]:
data['Gender_icon'] = data['Gender'].map({ 'F': 0,  'M' : 1})

In [ ]:
data.head()

,Id,Name,Year,Gender,Count,Gender_icon
0,1,Mary,1880,F,7065,0
1,2,Anna,1880,F,2604,0
2,3,Emma,1880,F,2003,0
3,4,Elizabeth,1880,F,1939,0
4,5,Minnie,1880,F,1746,0


TOKENIZATION

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd


# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to encode names
def encode_names(names, max_len=32):
    input_ids = []
    attention_masks = []

    for name in names:
        encoded = tokenizer.encode_plus(
            name,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=max_len,  # Pad or truncate
            padding='max_length',  # Pad to max_length
            truncation=True,  # Truncate to max_length
            return_attention_mask=True,
            return_tensors='pt'  # Return PyTorch tensors
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Encoding the labels
def encode_labels(genders):
    encoder = LabelEncoder()
    labels = encoder.fit_transform(genders)
    labels = torch.tensor(labels)
    return labels

# Encode the dataset
max_len = 64  # Maximum token length
input_ids, attention_masks = encode_names(data['Name'], max_len)
labels = encode_labels(data['Gender'])
labels = labels.long() 
# Create a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a DataLoader
batch_size = 2  # Example batch size
dataloader = DataLoader(
    dataset,
    sampler=RandomSampler(dataset),
    batch_size=batch_size
)



LOSS , OPTIMIZER AND MODEL

In [38]:
from transformers import BertForSequenceClassification, AdamW

# Load the pre-trained BERT model for classification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define the loss
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\UGBOKE GEORGE\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TRAINING LOOP USING BERT

In [40]:
from torch.utils.data import DataLoader

# DataLoader for handling batches
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
num_epochs = 3
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_masks, labels = batch
        
        # Zero the gradients before running the model
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")


In [ ]:
torch.save(model.state_dict(),'C:\Users\UGBOKE GEORGE\Downloads\NationalNames\N_gram-name-prediction\model_weigth.pth')

In [ ]:
torch.save(model,'C:\Users\UGBOKE GEORGE\Downloads\NationalNames\N_gram-name-prediction\complete_model.pth')

In [ ]:
train_loader = Dataloader(dataset, batch_size = 32, shuffle = True)
model.train()
for epoch in range (epoch_number):
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_masks, labels = batch
        optimizer.zero_grad()
        